In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
wd = '/cluster/work/borgw/SPSS/MultiOmicsAnalysis/ConsensusClustering/'
os.chdir(wd)

In [ ]:
def F_stat(view, clusters, k):
    data = copy.deepcopy(view)
    data['cluster'] = list(clusters)
    nj = data.groupby('cluster').size()
    overall_mean = data.drop('cluster', 1).mean()
    group_mean = data.groupby('cluster').mean()
    
    SSB = []
    for k in sorted(data['cluster'].unique()):
        diff = (group_mean.mean() - overall_mean) ** 2
        nj = data.groupby('cluster').size()[k]
        diff = nj * diff
        SSB.append(diff)
    SSB = pd.DataFrame(SSB).sum(0)
    
    SSW = pd.DataFrame(columns=group_mean.columns)
    for k in sorted(data['cluster'].unique()):
        cluster = data[data['cluster'] == k].drop('cluster', 1)
        SSW.loc[k] = ((cluster - group_mean.loc[k]) ** 2).sum(0)

    SSW = SSW.sum(0)
    N = len(data)
    coef = ((N - k) / (k - 1))
    
    F_stat = coef * (SSB/SSW)
    
    return F_stat

# clinical view

In [ ]:
clinical = pd.read_csv('data/ClinicalViewStandardized_binary_pca.csv', index_col=0)
clinical_ori = pd.read_csv('data/ClinicalViewMICEimputed.csv', index_col=0)
clinical_assignments = pd.read_csv('data/clinicalViewAssignments.csv', index_col=0)['3']
F_stat_clinical = F_stat(clinical, clinical_assignments, 3)

In [ ]:
F_stat_clinical.to_csv('F_stats/clinical.csv')

In [ ]:
clinical_ori['cluster'] = clinical_assignments.tolist()
stat = clinical_ori.groupby('cluster').mean().round(3).astype(str) + ' (' + clinical_ori.groupby('cluster').std().round(3).astype(str) + ')'
print(F_stat_clinical.sort_values(ascending=False).iloc[:10])
stat.loc[:, stat.columns.isin(F_stat_clinical.sort_values(ascending=False).iloc[:10].index)]

# proteome view

In [ ]:
proteome = pd.read_csv('data/ProteomeViewStandardized.csv', index_col=0)
proteome_ori = pd.read_csv('data/ProteomeView.csv', index_col=0)
proteome_assignments = pd.read_csv('data/ProteomeViewAssignments.csv', index_col=0)['3']
F_stat_proteome = F_stat(proteome, proteome_assignments, 3)
F_stat_proteome.to_csv('F_stats/proteome.csv')

In [ ]:
F_stat_proteome.sort_values(ascending=False).iloc[:10]

In [ ]:
npx = pd.read_excel('../olinks/20191053_Giannoni_NPX.xlsx', index_col=0, skiprows=[0, 1])
names = F_stat_proteome.dropna().sort_values(ascending=False).iloc[:10].index
uniprot = npx.loc[:, npx.columns.isin(names)].loc['Uniprot ID'].loc[names]
print(uniprot)

In [ ]:
proteome_ori['cluster'] = proteome_assignments.tolist()
stat = proteome_ori.groupby('cluster').mean().round(3).astype(str) + ' (' + proteome_ori.groupby('cluster').std().round(3).astype(str) + ')'

stat = stat.loc[:, stat.columns.isin(F_stat_proteome.sort_values(ascending=False).iloc[:10].index)]
stat.columns = uniprot.tolist()
stat

# two views

In [ ]:
view1 = pd.read_csv('data/ClinicalViewStandardized_binary_pca.csv', index_col=0)
view2 = pd.read_csv('data/ProteomeViewStandardized.csv', index_col=0)
view = pd.concat([view1, view2], 1)
true_assignments = pd.read_csv('data/ClinicalProteomeViewsAssignments.csv', index_col=0)['3']
F_stat_two = F_stat(view, true_assignments, 3)
F_stat_two.to_csv('F_stats/TwoViews.csv')

In [ ]:
F_stat_two.sort_values(ascending=False).iloc[:10]

In [ ]:
names = F_stat_two.dropna().sort_values(ascending=False).iloc[:10].index
uniprot = npx.loc[:, npx.columns.isin(names)].loc['Uniprot ID'].loc[names]
print(uniprot)

In [ ]:
proteome_ori['cluster'] = true_assignments.tolist()
stat = proteome_ori.groupby('cluster').mean().round(3).astype(str) + \
' (' + proteome_ori.groupby('cluster').std().round(3).astype(str) + ')'
stat = stat.loc[:, stat.columns.isin(F_stat_two.sort_values(ascending=False).iloc[:10].index)]
stat.columns = uniprot.tolist()
stat